In [65]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [66]:
from elasticsearch import Elasticsearch

In [67]:
es_client = Elasticsearch('http://localhost:9200')

In [68]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [69]:
index_name = 'intro-homework'

In [70]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'intro-homework'})

In [71]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [72]:
query = "How do I execute a command in a running docker container?"

In [73]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [74]:
response = es_client.search(index = index_name, body = search_query)

In [75]:
response['hits']['hits'][0]['_score']

84.050095

In [76]:
search_query_2 = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index = index_name, body = search_query_2)
response['hits']['hits']

[{'_index': 'intro-homework',
  '_id': 'CNWbb5ABQhGpST70bVs-',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'intro-homework',
  '_id': 'J9Wbb5ABQhGpST70bVtm',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_d

In [77]:
context_template = ''

In [78]:
for doc in response['hits']['hits']:
    context_template = context_template + f'Q: {doc['_source']['question']}\nA: {doc['_source']['text']}\n\n'

In [79]:
print(context_template)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [80]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [82]:
prompt = prompt_template.format(question = query, context = context_template)

In [87]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [109]:
print(len(prompt.strip()))

1462


In [90]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [110]:
len(encoding.encode(prompt.strip()))

322

In [111]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [113]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
model = 'gpt-4o',
messages = [{'role' : 'user', 'content' : prompt}])
print(response.choices[0].message.content)

To execute a command in a running Docker container, you can use the `docker exec` command. Here’s how:

1. First, find the container ID of the running container by listing all the running containers:
   ```bash
   docker ps
   ```

2. Then, use the following command to execute a command in the specific container:
   ```bash
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of the container you want to access.

This will start an interactive session with a bash shell in the running container.
